## 1. Create Anyscale Resources with Terraform

Set up the necessary Terraform variables and apply the configuration:

### Creating Anyscale Resources

Steps for deploying Anyscale resources via Terraform:

* Review and modify [variables.tf](variables.tf) with your configurations
* (Optional) Create a `terraform.tfvars` file to override any defaults
* View [main.tf](main.tf) to see how the resources are created


In [ ]:
"""
Set the global variables for the deployment, which are the same as in the variables.tf file.
"""

ANYSCALE_CLOUD_NAME = "anyscale-cloud-ec2-private-example" # Replace with your actual Anyscale cloud name
AWS_REGION = "us-west-2"  # Replace with your actual AWS region
ANYSCALE_S3_BUCKET_NAME = "anyscale-ec2-new-us-west-2" # You can leave it empty for now, you can get it from the output of the terraform apply command, then you can populate it here.

In [ ]:
# Run Terraform commands

# Initialize Terraform
!terraform init

# Preview the changes
!terraform plan

# Apply the changes
!terraform apply -auto-approve


<details>
<summary>Sometimes, your AWS account may have too many existing resources consuming capacity, causing you to hit service limits. In such cases, it's a good idea to release unused resources—such as unassociated Elastic IPs (EIPs) or idle NAT gateways.
</summary>
During you installing, you may need to release the unused EIPs and deleted unused NAT.

To find the unattached EIPs:
```bash
aws ec2 describe-addresses | jq '.Addresses[] | select(.InstanceId == null and .NetworkInterfaceId == null)'
```
You will see a list of unattached EIPs.

To release one:

```bash
aws ec2 release-address --allocation-id eipalloc-xxxxxx
```

To find the unused NAT in your region, for example us-west-2:
```bash
aws ec2 describe-nat-gateways --region us-west-2 --filter "Name=state,Values=available" | jq '.NatGateways | length'
```

To delete one:

```bash
aws ec2 delete-nat-gateway --nat-gateway-id nat-xxxxxxxx
```

To identify VPCs that look like they might be safe to delete (test/development ones):
```bash
aws ec2 describe-vpcs --query 'Vpcs[*].[VpcId,Tags[?Key==`Name`].Value|[0]]' --output table | grep -E "(test|temp|dev|scratch|derp|floral|scrumptious)"
```

to delete one:
```bash
aws ec2 delete-vpc --vpc-id vpc-0f8bb12ddf9a451e9  # derp-vpc-derp-sunbeam
```

You may need to delete more depends on your scenario.

</detail>

<div class="alert alert-block alert-info">
<b>Take a note to the output of terraform apply! </b>You will need it when you register the Anyscale cloud to your cloud provider.
</div>

<details>
<summary>Sample output</summary>
```
Apply complete! Resources: 50 added, 0 changed, 0 destroyed.

Outputs:

anyscale_registration_command = <<EOT
anyscale cloud register --provider aws \
  --name anyscale-cloud-ec2-private-example \
  --region us-west-2 \
  --vpc-id vpc-0xxxxxxxxx \
  --subnet-ids subnet-xxxxxxxxxx,subnet-xxxxxxxxxxxx,subnet-0xxxxxxxxxx,subnet-0xxxxxxxx \
  --s3-bucket-id anyscale-ec2-new-us-west-2 \
  --anyscale-iam-role-id arn:aws:iam::959243851260:role/anyscale-ec2-role-new \
  --instance-iam-role-id arn:aws:iam::959243851260:role/anyscale-cluster-node-xxxxxxxxxxxx \
  --security-group-ids sg-xxxxxxxxx \
  --efs-id fs-xxxxxxxxxx

EOT
efs_id = "fs-0bd7bde3f2c914b3e"
instance_profile_arn = "arn:aws:iam::xxxxx:instance-profile/anyscale-cluster-node-xxxxxxxxxx"
s3_bucket_id = "anyscale-ec2-new-us-west-2"
security_group_id = "sg-xxxxxxxxxx"
subnet_ids = [
  "subnet-0a000000000000000",
  "subnet-03000000000000000",
  "subnet-00000000000000000",
  "subnet-0a000000000000000",
]
vpc_id = "vpc-0000000000000"```
</details>